# Clustering Consulting Project - Solutions

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

**Imports**

In [46]:
import findspark
findspark.init('/home/sedat/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import ClusteringEvaluator

**Start spark session and read csv file**

In [3]:
spark = SparkSession.builder.appName('hack_kmeans').getOrCreate()

In [4]:
df = spark.read.csv('hack_data.csv', inferSchema=True, header=True)

In [6]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [7]:
df.count()

334

In [8]:
df.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

**Vector Assembler**

In [29]:
in_cols = df.columns

In [30]:
in_cols.remove('Location')
in_cols

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [33]:
assembler = VectorAssembler(inputCols=in_cols, outputCol='features')

In [35]:
assembler_df = assembler.transform(df)

**Standard Scaler**

In [38]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [39]:
scaled_df = scaler.fit(assembler_df).transform(assembler_df)

**Final data and model**

In [40]:
final_data = scaled_df.select('scaledFeatures')

In [42]:
final_data.show()

+--------------------+
|      scaledFeatures|
+--------------------+
|[0.56785108466505...|
|[1.41962771166263...|
|[2.20042295307707...|
|[0.14196277116626...|
|[1.41962771166263...|
|[0.07098138558313...|
|[1.27766494049636...|
|[1.56159048282889...|
|[1.06472078374697...|
|[0.85177662699757...|
|[1.06472078374697...|
|[2.27140433866020...|
|[1.63257186841202...|
|[0.63883247024818...|
|[1.91649741074455...|
|[0.85177662699757...|
|[1.49060909724576...|
|[0.70981385583131...|
|[1.41962771166263...|
|[1.56159048282889...|
+--------------------+
only showing top 20 rows



**Model and find out its 2 or 3!**

In [43]:
kmeans_2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans_3 = KMeans(featuresCol='scaledFeatures', k=3)

In [44]:
model_k2 = kmeans_2.fit(final_data)
model_k3 = kmeans_3.fit(final_data)

In [47]:
prediction_k2 = model_k2.transform(final_data)
prediction_k3 = model_k3.transform(final_data)

In [51]:
prediction_k2.show()

+--------------------+----------+
|      scaledFeatures|prediction|
+--------------------+----------+
|[0.56785108466505...|         1|
|[1.41962771166263...|         1|
|[2.20042295307707...|         1|
|[0.14196277116626...|         1|
|[1.41962771166263...|         1|
|[0.07098138558313...|         1|
|[1.27766494049636...|         1|
|[1.56159048282889...|         1|
|[1.06472078374697...|         1|
|[0.85177662699757...|         1|
|[1.06472078374697...|         1|
|[2.27140433866020...|         1|
|[1.63257186841202...|         1|
|[0.63883247024818...|         1|
|[1.91649741074455...|         1|
|[0.85177662699757...|         1|
|[1.49060909724576...|         1|
|[0.70981385583131...|         1|
|[1.41962771166263...|         1|
|[1.56159048282889...|         1|
+--------------------+----------+
only showing top 20 rows



**Evaluate clustering by computing Silhouette score model_k2**

In [52]:
evaluator = ClusteringEvaluator(featuresCol='scaledFeatures')
silhouette = evaluator.evaluate(prediction_k2)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.817646009401248


**Evaluate clustering by computing Silhouette score model_k3**

In [54]:
# Evaluate clustering by computing Silhouette score
silhouette = evaluator.evaluate(prediction_k3)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.7608455651454915


**Nothing definitive can be said with the above, but wait! The last key fact that the engineer mentioned was that the attacks should be evenly numbered between the hackers!**

In [56]:
model_k2.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [57]:
model_k3.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   88|
|         0|   79|
+----------+-----+



**It was 2 hackers, in fact, our clustering algorithm created two equally sized clusters with K=2, no way that is a coincidence!**